In [22]:
# !pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

In [23]:
# !git clone https://github.com/ultralytics/yolov5

In [24]:
# !pip install -r ./yolov5/requirements.txt

In [1]:
#install dependencies
import torch 
from matplotlib import pyplot as plt
import numpy as np
import cv2
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Using cache found in C:\Users\justi/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-11-9 Python-3.9.18 torch-2.0.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [4]:
img = "https://ultralytics.com/images/zidane.jpg"
results = model(img, size=640) 
results.pandas().xyxy[0].to_json(orient='records')

'[{"xmin":743.2904052734,"ymin":48.3436584473,"xmax":1141.7565917969,"ymax":720.0,"confidence":0.87986058,"class":0,"name":"person"},{"xmin":441.9896240234,"ymin":437.336730957,"xmax":496.5850830078,"ymax":710.0361938477,"confidence":0.6751193404,"class":27,"name":"tie"},{"xmin":123.0511474609,"ymin":193.2380981445,"xmax":714.6907348633,"ymax":719.7713012695,"confidence":0.6666929126,"class":0,"name":"person"},{"xmin":978.9898071289,"ymin":313.5794677734,"xmax":1025.3028564453,"ymax":415.526184082,"confidence":0.2615174651,"class":27,"name":"tie"}]'

In [5]:
# YOLOv5 🚀 by Ultralytics, AGPL-3.0 license
"""
Run a Flask REST API exposing one or more YOLOv5s models
"""

import argparse
import io

import torch
from flask import Flask, request
from PIL import Image

app = Flask(__name__)
models = {}

DETECTION_URL = '/v1/object-detection/<model>'


@app.route(DETECTION_URL, methods=['POST'])
def predict(model):
    if request.method != 'POST':
        return

    if request.files.get('image'):
        # Method 1
        # with request.files["image"] as f:
        #     im = Image.open(io.BytesIO(f.read()))

        # Method 2
        im_file = request.files['image']
        im_bytes = im_file.read()
        im = Image.open(io.BytesIO(im_bytes))

        if model in models:
            results = models[model](im, size=640)  # reduce size=320 for faster inference
            return results.pandas().xyxy[0].to_json(orient='records')


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Flask API exposing YOLOv5 model')
    parser.add_argument('--port', default=5000, type=int, help='port number')
    parser.add_argument('--model', nargs='+', default=['yolov5s'], help='model(s) to run, i.e. --model yolov5n yolov5s')
    opt = parser.parse_args()
    m = torch.hub.load('ultralytics/yolov5', 'yolov5s')
#     for m in opt.model:
    models[0] = torch.hub.load('ultralytics/yolov5', m, force_reload=True, skip_validation=True)

    app.run(host='0.0.0.0', port=opt.port)  # debug=True causes Restarting with stat


usage: ipykernel_launcher.py [-h] [--port PORT] [--model MODEL [MODEL ...]]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"8b3b899c-2443-4f47-b07d-d8047368308c" --shell=9002 --transport="tcp" --iopub=9004 --f=c:\Users\justi\AppData\Roaming\jupyter\runtime\kernel-v2-7700QLZADvn1oXS8.json


SystemExit: 2